## Scrap Finviz table using read_html

In [ ]:
#libraries to import 
import os
import sys
import time
import requests
import pandas as pd
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [ ]:
#add header and links
method_url_dict = {"Write all links with their corresponding keys where the table exists."}
headers = { "Write Header"}

#for multi retries
session = requests.Session()
retry = Retry(connect=3, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [ ]:
# Main function for scraping tables, link by link.
def getFullResponse(url, count=0):
    response = requests.get(url, headers=headers)
    # df = pd.read_html(response.text, attrs={'class': 'styled-table-new is-rounded is-tabular-nums w-full screener_table'})
    try:
        df = pd.read_html(response.text, attrs={'class': 'styled-table-new is-rounded is-tabular-nums w-full screener_table'})
    except ValueError as e:
        print(f"Error while parsing tables: {e}")
        return None
    tickers = df[0]['Ticker']
    all_tickers = []
    for ticker in tickers:
        ticker1.append(ticker)
    return all_tickers

In [ ]:
#Get all the data and transform it as needed.
def ingestScreener(database_name, symbols_in_db):
    try:
        result = {}
        count = 0
        total_count = 0
        for method, url in method_url_dict.items():
            range = 1
            result[method] = []
            while True:
                symbols = getFullResponse(f"{url}&r={range}")
                if symbols and len(symbols) > 1:
                    for symbol in symbols:
                        total_count += 1
                        #if symbol available update it.
                        if symbol in symbols_in_db:
                            count += 1
                            if symbol in result[method]:
                                break
                            else:
                            #else append symbol too
                                result[method].append(symbol)  
                    info_logger.log(f"symbols exist in db {count} out of {total_count} in in {time.time() - start:.2f} seconds")       
                else:
                    break
                range += 20

        symbols = set()
        for key, values in result.items():
            symbols.update(values)
        data = []
        columns = list(result.keys())
        columns.insert(0, 'symbol')
        for symbol in symbols:
            row = [symbol]
            for key in columns[1:]:
                if symbol in result.get(key, []):
                    row.append(True)
                else:
                    row.append(False)
            data.append(row)
        if data:
            db.truncateTable(database_name, table_name)
            success_logger.log(f"Successfully truncate data in database in {time.time() - start:.2f} seconds")
            db.insert(database_name=database_name, table_name=table_name, columns=columns, values=data)
            success_logger.log(f"Successfully saved data to database in {time.time() - start:.2f} seconds")

    except Exception as e:
        error_logger.log(e)

In [ ]:
if __name__ == "__main__":
    try:
        """
            Script Args:
                database_name: see database file for names
        """
        
        start = time.time()
        #To add an argument when running a script, we use sys.argv to pass the argument.
        list_operations = sys.argv
        database_name = list_operations[1]
        #to get symbols from database
        query = """
                    SELECT symbol FROM stock_symbols
                """
        symbols = db.getDataFromQuery(database_name=database_name, query=query)
        success_logger.log(f"Successfully get data from database {time.time() - start:.2f} seconds")
        symbols = [symbol[0] for symbol in symbols]
        ingestScreener(database_name, symbols)
        success_logger.log(f'Successfully ingested in {time.time() - start:.2f} seconds')
    except Exception as e:
        error_logger.log(e)